In [1]:
from collections import Counter
from scipy import spatial
import joblib
from sklearn.metrics.pairwise import euclidean_distances

import imp, multiprocessing
import LDA_ETM as lda
import glob
import os
import numpy as np
import copy
import pandas as pd
import pickle
import utils as my_utils
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [2]:
folder = "dumps/mrf_lda/amazon_*50*"

In [3]:
def get_diversity_score(list):

    all_words = []

    for l in list:

        all_words += l

    temp = pd.DataFrame(all_words,columns=['a'])

    temp = temp['a'].value_counts().reset_index()

    temp.columns = ['word','tot_cnt']

    return temp[temp.tot_cnt == 1].shape[0]*1.0/temp.tot_cnt.sum()

# Evaluations

In [4]:
dumps = glob.glob(folder)

In [5]:
dumps.sort(reverse=True)

In [6]:
dumps

['dumps/mrf_lda/amazon_movies_20000_glove_0.6_50topics',
 'dumps/mrf_lda/amazon_kindle_20000_glove_0.6_50topics',
 'dumps/mrf_lda/amazon_home_20000_glove_0.6_50topics']

In [9]:
def get_scores(sampler):

    dt_distribution = sampler.theta()

    ss = silhouette_score(euclidean_distances(sampler.matrix),
                          dt_distribution.argmax(axis=1), metric='precomputed')

    dbs = davies_bouldin_score(sampler.matrix, dt_distribution.argmax(axis=1))

    chs = my_utils.coherence_score(sampler.matrix, list(sampler.getTopKWords(10, sampler.words).values()), sampler.vocabulary)

    hsc = my_utils.get_hscore_multi(dt_distribution, sampler.matrix, sampler.n_topics, 2000)

    loli = sampler.loglikelihood()

    pxy = sampler.perplexity()
    
    chs2 = my_utils.coherence_score2(sampler.matrix, list(sampler.getTopKWords(10, sampler.words).values()), sampler.vocabulary)
    
    div = get_diversity_score(list(sampler.getTopKWords(10, sampler.words).values()))
    
    print(str(hsc) + "," + str(ss) + "," + str(dbs) + "," + str(chs) + "," + str(loli) + "," + str(pxy) + "," + str(chs2) + "," + str(div))

In [10]:
for i in dumps:
    print(i)
    get_scores(sampler = joblib.load(i))

dumps/mrf_lda/amazon_movies_20000_glove_0.6_50topics
0.5642029984429587,-0.06483622893014271,13.29183169074436,-127.09362795162397,-2942715.439212765,23489.67089741538,-4.098540749223562,1.0
dumps/mrf_lda/amazon_kindle_20000_glove_0.6_50topics
0.5385896604449018,-0.06754959973552477,13.438293758180837,-121.97279233113937,-2681931.3187743444,17169.364207770508,-3.9041531942018093,1.0
dumps/mrf_lda/amazon_home_20000_glove_0.6_50topics
0.5178474109116609,-0.06789467762060722,10.248945935040362,-128.8824496045749,-2875711.7365965103,15445.722398864924,-2.764778231742038,1.0


# ETM

In [7]:
sampler = joblib.load(dumps[0])

In [18]:
top_words = list(sampler.getTopKWords(10, sampler.words).values())

In [19]:
Counter(list(Counter(np.array(top_words).reshape(-1)).values()))

Counter({1: 500})

In [20]:
top_words

[['festival',
  'bergman',
  'kirk',
  'voyager',
  'douglas',
  'buffy',
  'ustinov',
  'pirate',
  'spike',
  'trek'],
 ['prejudice',
  'poirot',
  'diary',
  'pride',
  'gabriel',
  'davy',
  'heston',
  'charlton',
  'explosion',
  'walken'],
 ['zoom',
  'mediocre',
  'possibility',
  'tight',
  'integral',
  'reluctant',
  'shirt',
  'gate',
  'potty',
  'delete'],
 ['beatles',
  'black',
  'manga',
  'darth',
  'lagoon',
  'vader',
  'funimation',
  'shyamalan',
  'courtney',
  'anime'],
 ['larry',
  'invisible',
  'locate',
  'region',
  'election',
  'lewis',
  'abbott',
  'costello',
  'profit',
  'jerry'],
 ['ford',
  'harrison',
  'selleck',
  'terminator',
  'hayao',
  'matrix',
  'anderson',
  'airport',
  'bale',
  'christian'],
 ['rodriguez',
  'vote',
  'president',
  'leary',
  'political',
  'quentin',
  'cultural',
  'culture',
  'iraq',
  'tarantino'],
 ['ricci',
  'novak',
  'smash',
  'rita',
  'jesse',
  'astaire',
  'shaun',
  'dance',
  'ballet',
  'dancer'],
 

# Appendix

In [ ]:
# dt_dist = sampler.theta()

# X_embedded = TSNE(n_components=2).fit_transform(dt_dist)

# import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE

# plt.figure(figsize=(8, 8))
# plt.scatter([i[0] for i in X_embedded], [i[1] for i in X_embedded], c=dt_dist.argmax(axis=1))
# plt.legend(loc=2)
# plt.show()

In [ ]:
#     print ""
#     print "Topics:", k
#     print "Coherance:", my_utils.coherence_score(count_matrix, top_words, vocabulary)
#     print "Silhouette Score:", silhouette_score(count_matrix, dt_distribution.argmax(axis=1))
#     print "Davies Bouldin Score:", davies_bouldin_score(count_matrix, dt_distribution.argmax(axis=1))